# CTA Service Map -- Safety Department Incidents
## Author: Kenny Wang
### Date
- **Created: 9/16/2025**
- **Last Updated: 10/7/2025**
#### References:
- **NTD**: https://data.transportation.gov/Public-Transit/Major-Safety-Events/9ivb-8ae9/about_data
- GIS Data

In [1]:
# Import all libraries here
import pandas as pd
import folium
import geopandas as gpd
from shapely.geometry import LineString, MultiLineString
from pyproj import Transformer
import re
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext

## Create the base map of the CTA bus and rail

In [2]:
# 1. Load Bus Stops data from GIS
gis_bus = pd.read_excel("~/Desktop/CTA_Safety_Map/Base_map_data/Location Codes GIS.xlsx", sheet_name="Bus Stop Coordinates")
bus_stops = gis_bus.rename(columns={
    "PUBLIC_NAME": "Stop_Name",
    "POINT_X": "X",
    "POINT_Y": "Y"
})[["Stop_Name", "X", "Y", "ROUTESSTPG", "DIR"]]

# Convert from EPSG:3435 → EPSG:4326
transformer = Transformer.from_crs("EPSG:3435", "EPSG:4326", always_xy=True)
bus_stops["Longitude"], bus_stops["Latitude"] = transformer.transform(
    bus_stops["X"].values, bus_stops["Y"].values
)

# Display
bus_stops.head()

,Stop_Name,X,Y,ROUTESSTPG,DIR,Longitude,Latitude
0,Broadway & Wellington,1.171708e+06,1.920233e+06,36,SB,-87.644359,41.936578
1,69th Street & State (Red Line),1.177405e+06,1.859249e+06,169,EB,-87.625268,41.769107
2,Irving Park & Western,1.159741e+06,1.926461e+06,80,EB,-87.688164,41.953924
3,Racine & Roosevelt,1.168577e+06,1.894947e+06,60,NB,-87.656595,41.867262
4,State & 47th Street,1.177095e+06,1.873813e+06,29,NB,-87.625967,41.809079


In [3]:
# 2. Load Rail Stations + Routes/Lines with NTD data
rail_stations = pd.read_csv("~/Desktop/CTA_Safety_Map/Base_map_data/CTA_RailStations_CoordsFixed.csv")
rail_lines = pd.read_csv("~/Desktop/CTA_Safety_Map/Base_map_data/CTA_RailLines_Parsed.csv")

rail_stations["Latitude"] = rail_stations["Latitude_fixed"]
rail_stations["Longitude"] = rail_stations["Longitude_fixed"]



In [4]:
# 3. Load Bus Routes from Shapefile from GIS
bus_routes_gdf = gpd.read_file("~/Desktop/CTA_Safety_Map/Base_map_data/Bus Routes/CTA_BUSROUTES.shp")
print(f"Loaded {len(bus_routes_gdf)} bus routes from shapefile")
if bus_routes_gdf.crs != "EPSG:4326":
    bus_routes_gdf = bus_routes_gdf.to_crs(epsg=4326)

Loaded 127 bus routes from shapefile


In [5]:
# 4. Helper: Parse Route/Line Coordinates
def parse_coords(coord_str):
    coords = []
    for pair in str(coord_str).split(" "):
        if not pair.strip():
            continue
        try:
            lon, lat, *_ = pair.split(",")
            coords.append([float(lat), float(lon)])
        except:
            continue
    return coords

In [6]:
# 5. Create Base Map
m = folium.Map(location=[41.85, -87.65], zoom_start=11, tiles=None)
folium.TileLayer("CartoDB positron", name="Base Layer").add_to(m)

In [7]:
# 6. Feature Groups - layers
fg_bus_routes = folium.FeatureGroup(name="Bus Routes", show=True).add_to(m)
fg_rail_lines = folium.FeatureGroup(name="Rail Lines", show=True).add_to(m)
fg_bus_stops = folium.FeatureGroup(name="Bus Stops", show=False).add_to(m)
fg_rail_stations = folium.FeatureGroup(name="Rail Stations", show=True).add_to(m)


In [8]:
# 7. Rail Lines
rail_colors = {
    "Red Line": "#C60C30",
    "Blue Line": "#00A1DE",
    "Green Line": "#009B48",
    "Brown Line": "#62361B",
    "Orange Line": "#F9461C",
    "Pink Line": "#E27EA6",
    "Purple Line": "#522398",
    "Yellow Line": "#F9E300"
}

# Function to normalize the line names
def normalize_line_name(name):
    for key in rail_colors.keys():
        if key in name:
            return key
    return name

# Plot the rail lines
for _, row in rail_lines.iterrows():
    coords = parse_coords(row["Coordinates"])
    if coords:
        norm_name = normalize_line_name(str(row["Segment_Name"]))
        color = rail_colors.get(norm_name, "#444")
        folium.PolyLine(
            coords,
            color=color,
            weight=4,
            opacity=0.9,
            tooltip=row["Segment_Name"]
        ).add_to(fg_rail_lines)


In [9]:
# 8. Rail Stations (popup + rotated label)
# Plot the into the map
for _, row in rail_stations.drop_duplicates(subset=["Station_ID"]).iterrows():
    popup_html = f"""
    <b>{row['Station_Name']}</b><br>
    Long Name: {row['Station_LongName']}<br>
    ID: {row['Station_ID']}<br>
    Address: {row['Address']}<br>
    Line: {row['Line']}<br>
    ADA Accessible: {row['ADA_Accessible']}
    """

    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=4,
        color="black",
        fill=True,
        fill_color="white",
        fill_opacity=1,
        weight=1,
        popup=folium.Popup(popup_html, max_width=300),
        tooltip=row["Station_Name"]
    ).add_to(fg_rail_stations)

    folium.map.Marker(
        [row["Latitude"] + 0.0007, row["Longitude"] + 0.0007],
        icon=folium.DivIcon(
            html=f"""
                <div style="font-size: 12px; 
                            color:{rail_colors.get(row['Line'], 'black')};
                            transform: rotate(-30deg);
                            white-space: nowrap;
                            text-shadow: 1px 1px 2px white;">
                    {row['Station_Name']}
                </div>
            """
        )
    ).add_to(fg_rail_stations)
    

In [10]:
# 9. Bus Routes (dark gray from shapefile)
from shapely.geometry import MultiLineString, LineString

for _, row in bus_routes_gdf.iterrows():
    geom = row.geometry
    route_name = f"{row.ROUTE} - {row.NAME}"

    if isinstance(geom, LineString):
        coords = [(lat, lon) for lon, lat in geom.coords]
        folium.PolyLine(coords, color="#555555", weight=2, opacity=0.8,
                        tooltip=route_name).add_to(fg_bus_routes)
    elif isinstance(geom, MultiLineString):
        for line in geom.geoms:
            coords = [(lat, lon) for lon, lat in line.coords]
            folium.PolyLine(coords, color="#555555", weight=2, opacity=0.8,
                            tooltip=route_name).add_to(fg_bus_routes)


In [11]:
# 10. Bus Stops
for _, row in bus_stops.iterrows():
    popup_text = f"<b>{row['Stop_Name']}</b><br>Routes: {row['ROUTESSTPG']}<br>Dir: {row['DIR']}"
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=2,
        color="green",
        fill=True,
        fill_color="green",
        fill_opacity=0.3,
        tooltip=popup_text
    ).add_to(fg_bus_stops)


## Match and plot the incidenets

### Safeline

In [12]:
# SharePoint credentials
site_url = "https://transitchicago.sharepoint.com/sites/Safety-SMS"
# Should always replace the valid credentials
username = "JWang.int@transitchicago.com"
password = "Wjk0207230012?!"

# Connect
ctx = ClientContext(site_url).with_credentials(UserCredential(username, password))

# Path to Excel file inside SharePoint
file_url = "/sites/Safety-SMS/Shared Documents/SafeLine/2025/Safeline Tracking Log 2025.xlsx"

# Download the Excel file locally
download_path = "Safeline_2025.xlsx"
with open(download_path, "wb") as output_file:
    response = ctx.web.get_file_by_server_relative_url(file_url).download(output_file).execute_query()

print("File downloaded!")

# Read 'Master' sheet
safeline_df = pd.read_excel(download_path, sheet_name="Master Safeline 2025")
safeline_df.head()

File downloaded!


C:\Users\JWang.int\AppData\Local\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


,Name,Reported From,"Report number (YYYY,MM, Report #)",Date of Event,Date of Event Report Received in Safety,Time of the Hazard Event,Caller Information,Description,Safety Concern,Location,...,Unnamed: 16372,Unnamed: 16373,Unnamed: 16374,Unnamed: 16375,Unnamed: 16376,Unnamed: 16377,Unnamed: 16378,Unnamed: 16379,Unnamed: 16380,Unnamed: 16381
0,Jessica Brant,Safeline,202501-01,2025-01-02,2025-01-02 00:00:00,7:35 Hours,Alan Moss,\n\n\nCTA employee indicating that at approxim...,Lights giving out at the Pulaski station.,Greenline,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Jessica Brant,Safeline,202501-02,2025-01-08,2025-01-09 00:00:00,10:57 hours,773-932-3509,An employee has raised several safety concerns...,Defective (Unstable) ladder and harness for Gr...,Pink line,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Jessica Brant,Safeline,202501-03,2025-01-15,2025-01-15 00:00:00,12:36hours,7736644394,Bus operator states none of the buses have win...,Buses not having windshieldld wiper fluid,Unknown,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Jessica Brant,Safeline,202501-04,2025-01-16,2025-01-16 00:00:00,8:46hours,8478466625,Plumber is reviewing the toolbox talk document...,No safety concern. Medical benefit concern.,Uknown,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Jessica Brant,Safeline,202501-05,2025-01-18,2025-01-21 00:00:00,01:00 hours,312-292-0311,There’s a tree branch that’s sticking out of t...,Broken tree branch on bus route that can poten...,6442 W Higgins Ave WB (Bus route 88 Higgins),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Normalize text
safeline_df["Combined_Text"] = (
    safeline_df["Location"].fillna("") + " " + safeline_df["Safety Concern "].fillna("")
).str.lower()

bus_stops["Stop_Name_Norm"] = bus_stops["Stop_Name"].str.strip().str.lower()
rail_stations["Station_Name_Norm"] = rail_stations["Station_Name"].str.strip().str.lower()


In [14]:
# Match Helper
def match_keyword(text, keyword_list):
    if not isinstance(text, str):
        return None
    for word in re.findall(r'\b[a-z]{4,}\b', text):
        for name in keyword_list:
            if word in name:
                return name
    return None


In [15]:
# Match Bus + Rail
safeline_df["Matched_Bus_Stop"] = safeline_df["Combined_Text"].apply(
    lambda x: match_keyword(x, bus_stops["Stop_Name_Norm"])
)
bus_matches = safeline_df[safeline_df["Matched_Bus_Stop"].notnull()]
bus_matches = bus_matches.merge(bus_stops, left_on="Matched_Bus_Stop", right_on="Stop_Name_Norm")

safeline_df["Matched_Rail_Station"] = safeline_df["Combined_Text"].apply(
    lambda x: match_keyword(x, rail_stations["Station_Name_Norm"])
)
rail_matches = safeline_df[safeline_df["Matched_Rail_Station"].notnull()]
rail_matches = rail_matches.merge(rail_stations, left_on="Matched_Rail_Station", right_on="Station_Name_Norm")

# Unmatched Reports
matched_locs = set(bus_matches["Location"]) | set(rail_matches["Location"])
unmatched = safeline_df[~safeline_df["Location"].isin(matched_locs)]
unmatched = unmatched.dropna(subset=["Safety Concern "])  # drop NaN concerns


In [16]:
# Safeline Layer (one combined)
fg_safeline = folium.FeatureGroup(name="Safeline Reports", show=True).add_to(m)

# Plot bus matches
for _, row in bus_matches.dropna(subset=["Safety Concern "]).iterrows():
    popup_html = f"""
    <b>Bus Stop:</b> {row['Stop_Name']}<br>
    <b>Concern:</b> {row['Safety Concern ']}
    """
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=6,
        color="darkblue",
        fill=True,
        fill_color="darkblue",
        fill_opacity=0.6,
        popup=folium.Popup(popup_html, max_width=300),
        tooltip="Safeline Concern"
    ).add_to(fg_safeline)


In [17]:
# Plot rail matches
for _, row in rail_matches.dropna(subset=["Safety Concern "]).iterrows():
    popup_html = f"""
    <b>Rail Station:</b> {row['Station_Name']}<br>
    <b>Concern:</b> {row['Safety Concern ']}
    """
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=6,
        color="darkblue",
        fill=True,
        fill_color="darkblue",
        fill_opacity=0.6,
        popup=folium.Popup(popup_html, max_width=300),
        tooltip="Safeline Concern"
    ).add_to(fg_safeline)

# Plot unmatched as notepad (still inside Safeline layer)
unmatched_html = "<h4>Unmatched Safeline Reports</h4><ul>"
for _, row in unmatched.iterrows():
    unmatched_html += f"<li><b>Location:</b> {row['Location']}<br><b>Concern:</b> {row['Safety Concern ']}</li><br>"
unmatched_html += "</ul>"

folium.Marker(
    location=[41.9, -87.6],  # in lake
    icon=folium.Icon(color="orange", icon="info-sign"),
    popup=folium.Popup(unmatched_html, max_width=400),
    tooltip="Click to view unmatched reports"
).add_to(fg_safeline)


## Save to HTML map

In [18]:
# Final Layer Control
folium.LayerControl(collapsed=False).add_to(m)
# Save to HTML
m.save("index.html")
print("✅ Map saved to index.html")


✅ Map saved to CTA_Safety_Map.html
